In [ ]:
!pip install -r requirements.txt

In [1]:
# Import
import pandas as pd
import numpy as np

In [17]:
data = pd.read_csv('../raw_data/export_2024-03-18.csv')
data.head()

,source,theme,soustheme,title,description,link,published,image,added_date
0,Écran large,culture,cinema,Test Final Fantasy 7 Rebirth : enfin le grand ...,Quatre ans après la sortie de Final Fantasy VI...,https://www.ecranlarge.com/jeux-video/critique...,"Sun, 17 Mar 2024 20:00:00 +0100",NaN,2024-03-18 09:00:00
1,Écran large,culture,cinema,True Detective : on a classé toutes les saison...,"La saison 1, la saison 2, la saison 3 ou la sa...",https://www.ecranlarge.com/series/dossier/1504...,"Sun, 17 Mar 2024 19:15:00 +0100",NaN,2024-03-18 09:00:00
2,Écran large,culture,cinema,Marvel change de stratégie et relance (encore)...,Le producteur Brad Winderbaum a annoncé que Ma...,https://www.ecranlarge.com/films/news/1505616-...,"Sun, 17 Mar 2024 17:00:00 +0100",NaN,2024-03-18 09:00:00
3,Écran large,culture,cinema,Dune 2 : le scénariste justifie cette énorme d...,"Jon Spaihts, le scénariste de Dune: Deuxième P...",https://www.ecranlarge.com/films/news/1505614-...,"Sun, 17 Mar 2024 16:00:00 +0100",NaN,2024-03-18 09:00:00
4,Écran large,culture,cinema,Dans la peau de Blanche Houellebecq : critique...,"Vendu comme une comédie grasse, Dans la peau d...",https://www.ecranlarge.com/films/critique/1505...,"Sun, 17 Mar 2024 14:45:00 +0100",NaN,2024-03-18 09:00:00


# Création de la TABLE Category

In [18]:
data.theme.unique()
theme_list = ['culture', 'musique', 'technologie', 'politique', 'sport', 'economie',
         'environnement', 'general', 'emploi', 'cite', 'management', 'presse',
         'nustrale','tourisme']
data = data[data.theme.isin(theme_list)]
data.theme.unique()

array(['culture', 'musique', 'technologie', 'politique', 'sport',
       'economie', 'environnement', 'general', 'emploi', 'cite',
       'management', 'presse', 'nustrale', 'tourisme'], dtype=object)

In [19]:
cat_df = pd.DataFrame({'category_id': list(range(1, len(theme_list) + 1)),
        'cat_name': theme_list})
cat_df

,category_id,cat_name
0,1,culture
1,2,musique
2,3,technologie
3,4,politique
4,5,sport
5,6,economie
6,7,environnement
7,8,general
8,9,emploi
9,10,cite


In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

In [21]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
CAT_TABLE='category_dataset'
CAT_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{CAT_TABLE}' # the-mdr-project.live_mdr.category_dataset

In [13]:
client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
write_mode = 'WRITE_TRUNCATE'
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
job = client.load_table_from_dataframe(cat_df, CAT_TABLE_ID, job_config=job_config)
result = job.result()

## Test d'une query sur la category

In [14]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
key_path = "mathieu_credentials.json"

# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
CAT_TABLE='category_dataset'
CAT_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{CAT_TABLE}' # the-mdr-project.live_mdr.category_dataset

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
query = """SELECT *
            FROM the-mdr-project.live_mdr.category_dataset"""
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()
df

,category_id,cat_name
0,1,culture
1,2,musique
2,3,technologie
3,4,politique
4,5,sport
5,6,economie
6,7,environnement
7,8,general
8,9,emploi
9,10,cite


# Implémentation de la TABLE news

In [22]:
# Dataframe which is associate withe the news table
result = data.merge(cat_df, left_on='theme', right_on='cat_name')
result['news_id'] = np.arange(1, len(result) + 1)
news_df = result[['news_id', 'category_id', 'title', 'description', 'link','image', 'added_date', 'source', 'soustheme']]
news_df.rename(columns={'soustheme': 'sub_cat'}, inplace=True)
news_df

/var/folders/7_/qgxv4n054fz23g05r4yrv3kw0000gn/T/ipykernel_14250/2971487670.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df.rename(columns={'soustheme': 'sub_cat'}, inplace=True)


,news_id,category_id,title,description,link,image,added_date,source,sub_cat
0,1,1,Test Final Fantasy 7 Rebirth : enfin le grand ...,Quatre ans après la sortie de Final Fantasy VI...,https://www.ecranlarge.com/jeux-video/critique...,NaN,2024-03-18 09:00:00,Écran large,cinema
1,2,1,True Detective : on a classé toutes les saison...,"La saison 1, la saison 2, la saison 3 ou la sa...",https://www.ecranlarge.com/series/dossier/1504...,NaN,2024-03-18 09:00:00,Écran large,cinema
2,3,1,Marvel change de stratégie et relance (encore)...,Le producteur Brad Winderbaum a annoncé que Ma...,https://www.ecranlarge.com/films/news/1505616-...,NaN,2024-03-18 09:00:00,Écran large,cinema
3,4,1,Dune 2 : le scénariste justifie cette énorme d...,"Jon Spaihts, le scénariste de Dune: Deuxième P...",https://www.ecranlarge.com/films/news/1505614-...,NaN,2024-03-18 09:00:00,Écran large,cinema
4,5,1,Dans la peau de Blanche Houellebecq : critique...,"Vendu comme une comédie grasse, Dans la peau d...",https://www.ecranlarge.com/films/critique/1505...,NaN,2024-03-18 09:00:00,Écran large,cinema
...,...,...,...,...,...,...,...,...,...
440580,440581,11,Comment mesurer le management ?,<p>Certains considèrent que le management cons...,https://www.hbrfrance.fr/management/comment-me...,https://tra.img.pmdstatic.net/fit/~1~hbr~2024~...,2024-03-07 22:00:00,HBR,general
440581,440582,11,Les innovations managériales fondées sur le po...,<p>Alors que la troisième révolution industrie...,https://www.hbrfrance.fr/chroniques-experts/20...,https://tra.img.pmdstatic.net/fit/~1~hbr~2024~...,2024-03-07 22:00:00,HBR,general
440582,440583,11,Penser le futur du travail implique aussi de p...,<p>L'IA et la crise écologique transforment le...,https://www.hbrfrance.fr/strategie/penser-le-f...,https://tra.img.pmdstatic.net/fit/~1~hbr~2024~...,2024-03-07 22:00:00,HBR,general
440583,440584,11,Comment notre passion pour les réseaux sociaux...,<p>Dans un contexte où les utilisateurs produi...,https://www.hbrfrance.fr/magazine/2020/12/3231...,https://tra.img.pmdstatic.net/fit/~1~hbr~2024~...,2024-03-07 22:00:00,HBR,general


In [23]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
NEWS_TABLE='news_dataset'
NEWS_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{NEWS_TABLE}' # the-mdr-project.live_mdr.news_dataset

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

In [24]:
client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
write_mode = 'WRITE_TRUNCATE'
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
job = client.load_table_from_dataframe(news_df, NEWS_TABLE_ID, job_config=job_config)
result = job.result()

## Test de l'implementation avec une query

In [25]:
query = """SELECT *
            FROM the-mdr-project.live_mdr.news_dataset
            LIMIT 10"""
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()
df

,news_id,category_id,title,description,link,image,added_date,source,sub_cat
0,282,1,La France : le goût de l'engagement ? Envie d'...,"Jaleh Bradea reçoit Damien Baldin, directeur g...",https://www.youtube.com/watch?v=DGBT3awmaB8,https://www.youtube.com/v/DGBT3awmaB8?version=3,2024-03-18 09:00:00,C8 - la chaîne télé : youtube,None
1,295,1,Ludivine Sagnier : son espoir pour les nouvell...,Toutes les émissions de Clique sont à voir gra...,https://www.youtube.com/watch?v=qGuKKxWj_QU,https://www.youtube.com/v/qGuKKxWj_QU?version=3,2024-03-18 09:00:00,Canal Plus : Clique : youtube,None
2,296,1,Abrège Frère abrège le prix Goncourt Nicolas M...,Toutes les émissions de Clique sont à voir gra...,https://www.youtube.com/watch?v=bQ5q5RULoPM,https://www.youtube.com/v/bQ5q5RULoPM?version=3,2024-03-18 09:00:00,Canal Plus : Clique : youtube,None
3,297,1,« Passion : collectionneur de Barbie » à retro...,Le Plus de CANAL+ : Ce qu'il ne fallait pas ra...,https://www.youtube.com/watch?v=UPevKUNtVz4,https://www.youtube.com/v/UPevKUNtVz4?version=3,2024-03-18 09:00:00,Canal Plus : youtube,None
4,298,1,"""Je pourrais mourir une Barbie à la main"" l PA...","Jérôme, fan inconditionnel de Barbie, nous a o...",https://www.youtube.com/watch?v=pyzewq8P-GA,https://www.youtube.com/v/pyzewq8P-GA?version=3,2024-03-18 09:00:00,Canal Plus : youtube,None
5,312,2,LES PLUS BELLES CHANSONS CORSES ANTOINE CIOSI...,Chansons Corses Antoine Ciosi Nostalgia olivi ...,https://www.youtube.com/watch?v=t5YlGIKp1x4,https://www.youtube.com/v/t5YlGIKp1x4?version=3,2024-03-18 09:00:00,Chansons Corses Corsica Kallisté Coppelia Oliv...,None
6,700,2,"Michel Sardou blague sur le consentement, des ...",«Je vais vous chanter une chanson qu’on ne pou...,https://www.20minutes.fr/arts-stars/culture/mu...,https://img.20mn.fr/dgoN6rMxQgGHNI6sRta0FSk/14...,2024-03-18 09:00:00,Flipboard : musique,None
7,701,2,"Manuel Bompard, à propos de la chanteuse Aya N...","Manuel Bompard, coordinateur de la France Inso...",https://www.cnews.fr/videos/france/2024-03-17/...,https://ic-cdn.flipboard.com/cnews.fr/e1caf933...,2024-03-18 09:00:00,Flipboard : musique,None
8,702,2,La chanson préférée du prince George : un titr...,"Tel père, tel fils. Au détour d’une cérémonie ...",https://www.parismatch.com/royal-blog/la-chans...,https://ic-cdn.flipboard.com/parismatch.com/d1...,2024-03-18 09:00:00,Flipboard : musique,None
9,703,2,François Gemenne quitte la maison d’édition Fa...,François Gemenne quitte la maison d’édition Fa...,https://www.lesoir.be/575064/article/2024-03-1...,https://ic-cdn.flipboard.com/rosselcdn.net/4c1...,2024-03-18 09:00:00,Flipboard : musique,None


# Implémentation de la TABLE User

In [13]:
username = ['marru', 'dlouvegnies', 'rvincenti']
password = ['marrumdp', 'dlouvegniesmdp', 'rvincentimdp' ]

user_df = pd.DataFrame({'user_id': list(range(1, len(username) + 1)),
        'username': username,
        'password': password})
user_df

,user_id,username,password
0,1,marru,marrumdp
1,2,dlouvegnies,dlouvegniesmdp
2,3,rvincenti,rvincentimdp


In [14]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
USER_TABLE='user_dataset'
USER_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{USER_TABLE}' # the-mdr-project.live_mdr.user_dataset

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

In [15]:
client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
write_mode = 'WRITE_TRUNCATE'
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
job = client.load_table_from_dataframe(user_df, USER_TABLE_ID, job_config=job_config)
result = job.result()

,user_id,username,password
0,1,marru,marrumdp
1,2,dlouvegnies,dlouvegniesmdp
2,3,rvincenti,rvincentimdp


In [21]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
NEWS_TABLE='news_dataset'
NEWS_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{NEWS_TABLE}' # the-mdr-project.live_mdr.news_dataset

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)

query = """SELECT ROW_NUMBER() OVER() AS row_number, news_id, LENGTH(TO_JSON_STRING(t)) AS taille_ligne
            FROM the-mdr-project.live_mdr.news_dataset t
            ORDER BY taille_ligne DESC
            LIMIT 100
        """

job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

for row in query_job.result():
    print("Index:", row.row_number)
    print("ID de la news:", row.news_id)
    print("Taille de la ligne", row.taille_ligne)

Index: 295480
ID de la news: 137459
Taille de la ligne 484756
Index: 10060
ID de la news: 197123
Taille de la ligne 483854
Index: 10062
ID de la news: 197127
Taille de la ligne 391198
Index: 10065
ID de la news: 197126
Taille de la ligne 355091
Index: 10056
ID de la news: 197125
Taille de la ligne 354936
Index: 10064
ID de la news: 197124
Taille de la ligne 354891
Index: 12346
ID de la news: 249009
Taille de la ligne 332205
Index: 97287
ID de la news: 194991
Taille de la ligne 317549
Index: 392320
ID de la news: 259978
Taille de la ligne 283365
Index: 91546
ID de la news: 394124
Taille de la ligne 228390
Index: 92035
ID de la news: 154692
Taille de la ligne 227016
Index: 92696
ID de la news: 394236
Taille de la ligne 221053
Index: 92234
ID de la news: 61642
Taille de la ligne 214583
Index: 230273
ID de la news: 367765
Taille de la ligne 211877
Index: 51822
ID de la news: 409142
Taille de la ligne 198335
Index: 240088
ID de la news: 366240
Taille de la ligne 161679
Index: 14765
ID de la

In [20]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
NEWS_TABLE='news_dataset'
NEWS_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{NEWS_TABLE}' # the-mdr-project.live_mdr.news_dataset

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)

query = """
    DELETE FROM the-mdr-project.live_mdr.news_dataset
    WHERE LENGTH(TO_JSON_STRING(t)) > 500000
    """

job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)